## Module 4: Numerical Differentiation #

*Material for this worksheet can be found in the AE2220-I lecture notes Chapter 6.*

### Taylor-series construction of difference formulae

One very useful application of the Taylor Series is to approximate derivatives.  Once we can approximate derivatives we can start thinking about approximating Ordinary/Partial Differential Equations (ODEs/PDEs).

Recall the *Taylor series* expression:
$$
f(x + h) = f(x) + f'(x)h + \frac{f''(x)}{2}h^2 + ... + \frac{f^{k-1}(x)}{(k-1)!}h^{k-1} + R_k(\xi)
$$
where
$$
R_k(\xi) = \frac{f^k(\xi)}{k!}h^k, \hspace{1cm}  x < \xi < x+h
$$

Rearranging so that $f'$ is on the left, and everything else is on the right, we obtain:

$$
f'(x) = \frac{f(x+h)-f(x)}{h} + \frac{f''(x)}{2}h + ... +\frac{f^{k-1}(x)}{(k-1)!}h^{k-2} + O(h^{k-1})= \frac{f(x+h)-f(x)}{h} + O(h) 
$$

Once more the error is in terms of $h$, so by choosing $h$ small enough, the error can become as small as we like.

Now assume $f$ is sampled uniformly at points $x_i$ a distance $h = x_{i+1}-x_i$ apart.  Let $f_i := f(x_i)$, then the above becomes the *forward difference formula* (FDF):

$$
f'(x_i) =  \frac{f_{i+1}-f_i}{h} + O(h) 
$$

This is the simplest expression for the first derivative.  A different expression is obtained by using $x_{i-1}$; the *backward difference formula* (BDF): 

$$
f'(x_i) =  \frac{f(x)-f(x-h)}{h} + O(h) =\frac{f_i-f_{i-1}}{h} + O(h) 
$$

We introduce the *discrete derivative operator* $D_h$ - the numerical equivalent of $d/dx$:
$$
D_h\; f(x) := \frac{f(x+h) - f(x)}{h} \simeq \frac{d}{dx} f(x),
$$
where we used FDF to define $D_h$.

### Numerical experiments

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'axes.labelsize': 18})
import numpy as np

Let's assume we have the function $f(x)=\sin(x^2)$ in an interval [0,5].

In [ ]:
a, b = 0., 5.
h = 0.1         # spacing between consecutive points
n = (b-a)/h     # number of points n+1

def f(x): return np.sin(x**2)
def f_prime(x): return np.cos(x**2)*2*x

xi = np.linspace(a,b,n)          # Samples 
fi = f(xi)                       # Data 
xx = np.linspace(a,b,1001)       # Points for plotting

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(xx, f(xx))              # Plot fn
plt.plot(xi, fi, 'ok')
plt.xlabel(r'$x$'); plt.ylabel(r'$f$')
plt.subplot(122)             
plt.plot(xx, f_prime(xx))        # Plot derivative
plt.xlabel(r'$x$'); plt.ylabel(r"$f'$")

The computation of the forward difference is as follows:

In [ ]:
Df_fdf = (fi[1:] - fi[:-1]) / h   # Numerical derivatives
xi_fdf = xi[:-1]                  # Locations of derivatives 

Note that the derivative at point $x_n$ is not computed, as the data $x_{n+1}$ needed by the FDF is not available - hence `xi_fdf` describes only the first $n$ samples.  Let's examine the performance of the FDF:

In [ ]:
def plot_derivatives(xi, Df):
    plt.figure(figsize=(12,4))
    plt.subplot(121)
    plt.plot(xx, f_prime(xx), label='Exact')
    plt.plot(xi, Df, 'ok', label=r'$D_h f$')
    plt.xlabel(r'$x$'); plt.ylabel(r"$f'$")
    plt.legend(loc='best')
    plt.subplot(122)
    plt.plot(xi, Df - f_prime(xi), 'r', label=r'$D_h f$')
    plt.xlabel(r'$x$'); plt.ylabel(r"$\epsilon$")

In [ ]:
 plot_derivatives(xi_fdf, Df_fdf)

**Exercise 1:**
**(a) Modify the above code for FDF to get BDF (backward differences).  Be careful at boundaries - now there will be no derivative computed at sample $x_0$.  What do you notice about the expression for `Df_bdf` compared to `Df_fdf`? What do you notice about the error-plots compared to FDF?  What do you notive about the *sign* of the error?**

In [ ]:
### TODO
Df_bdf =                         # Numerical derivatives
xi_bdf =                         # Locations of derivatives 

In [ ]:
plot_derivatives(xi_bdf, Df_bdf)

**(b) What happens for changing $h$?  Theory says the error is linear in $h$, is this true? (Divide $h$ by 10, and see if the error reduces by about 10.)**

### Higher-order difference formulae

As always in this course, we want to obtain small errors as "quickly" as possible - i.e. with the fewest possible samples of $f$.  Specifically we would like $\epsilon = O(h^p)$ for some positive integer $p$.  The schemes above have $p=1$ - what we call "low"-order.  Taylor series so that higher order terms cancel out. For example, if we substract the series for the forward difference and the series for the backward difference: 

\begin{align}
f(x + h) &= f(x) + f'(x)h + \frac{f''(x)}{2}h^2 + \frac{f'''(x)}{3!}h^3 +... + R_k(\xi_1) \qquad \text{Forward}\\
f(x - h) &= f(x) - f'(x)h + \frac{f''(x)}{2}h^2 - \frac{f'''(x)}{3!}h^3 +... + R_k(\xi_2) \qquad \text{Backward}
\end{align}

Rearranging:

$$
f(x + h) - f(x - h) = 2f'(x)h + 2\frac{f'''(x)}{3!}h^3 + ... 
\hspace{0.5cm} \implies \hspace{0.5cm}
f'(x) = \frac{f(x + h) - f(x - h)}{2h} + O(h^2)
$$

For uniformly spaced samples $x_i$, we get the *central difference formula* (CDF): 

$$
f'(x_i) = \frac{f_{i+1} - f_{i-1}}{2h} + O(h^2)
$$

**Exercise 2:**
**(a) Again, modify the code for the forward difference to implement a central difference.  Now, only derivatives from $x_1,\dots,x_{n-1}$ will be computable.**

In [ ]:
### TODO
Df_cdf =                 # Numerical derivatives
xi_cdf =                 # Locations of derivatives 

In [ ]:
plot_derivatives(xi_cdf, Df_cdf)

**(b) Is the error lower compared to FDF and BDF?  If $h$ is reduced by a factor of 10, by how much is the error in CDF reduced? (Answer first based on theory, then by numerical experiment.)**  